# BigDataViewer in imglyb

Note that this notebook does not run on OSX! You can copy the contents of the relevant cells into a file and run the file through thw `OSWAWTwrapper`:
```bash
WRAPPER="$(python -c 'import site; print( site.getsitepackages()[0] )')/imglyb/OSXAWTwrapper.py"
python $WRAPPER /path/to/file
```
This example uses, in addition to imglyb, the *h5py* and *scikit-image* packages. You can install them through conda:
```bash
conda install h5py scikit-image
```

In [2]:
import h5py
import math
import numpy as np
import os
import pathlib
import skimage.io
import subprocess
import tempfile
import timeit
import threading
import time

## imglyb imports
An additional *tmp* directory within the classpath will allow us to compile Java code on the fly if we need fast pixel-wise method calls (see example below). 

In [3]:
tmp_dir = tempfile.mkdtemp()

import jnius_config
jnius_config.add_classpath( tmp_dir )

import imglyb
import imglyb.util as util

from jnius import autoclass, PythonJavaClass, java_method, cast

## Wait For BDV
Python is not aware of any Java threads and -- if not run in interactive mode -- would exit even with an active BDV window open. The `bdv_exit_check` as defined below keeps a Python thread alive while BDV is running and prevents Python from exiting. This is not necessary for interactive sessions (e.g. IPython shell, notebook) but is still added for the purpose of demonstration.

In [4]:
def bdv_exit_check( viewer, run_on_exit = lambda : print( "Exited!" ) ):

    check = autoclass( 'net.imglib2.python.BdvWindowClosedCheck' )()
    frame = cast( 'javax.swing.JFrame', autoclass( 'javax.swing.SwingUtilities' ).getWindowAncestor( viewer ) )
    frame.addWindowListener( check )

    def sleeper():
        while check.isOpen():
            time.sleep( 0.1 )
        run_on_exit()

    t = threading.Thread( target=sleeper )
    t.start()

## Multi-Channel vs ARGB

In [5]:
url = 'http://www.nerdtests.com/mq/testimages/167138_4f49b66c0cb4a87cc906.jpg'
# url = '/home/phil/Pictures/10pm.png'
arr = skimage.io.imread( url )
rai = imglyb.to_imglib( arr )
bdv = util.BdvFunctions.show( rai, 'argb' )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel() )

Exited!


In [6]:
rai = imglyb.to_imglib( np.transpose( arr, ( 2, 0, 1 ) ) )
bdv = util.BdvFunctions.show( rai, 'argb', util.BdvOptions.options().is2D() )
bdv_exit_check( bdv.getBdvHandle(). getViewerPanel() )

Exited!


## Pure Python Converter

In [7]:
ARGBType = autoclass( 'net.imglib2.type.numeric.ARGBType' )

class CompositeARGBConverter( PythonJavaClass ):
    
    __javainterfaces__ = [ 'net.imglib2.converter.Converter' ]
    
    def __init__( self ):
        super( CompositeARGBConverter, self ).__init__()
        
    @java_method( '(Ljava/lang/Object;Ljava/lang/Object;)V' )
    def convert( self, source, target ):
        target.set( ARGBType.rgba( 
            source.get( 0 ).getInteger(), 
            source.get( 1 ).getInteger(), 
            source.get( 2 ).getInteger(),
            255 ) )
        
       
Converters = autoclass( 'net.imglib2.converter.Converters' )
converter = CompositeARGBConverter()
composite = util.Views.collapseReal( rai )
view = Converters.convert( cast( 'net.imglib2.RandomAccessibleInterval', composite ), converter, ARGBType() )
img = autoclass( 'net.imglib2.img.array.ArrayImgs' ).argbs( view.dimension( 0 ), view.dimension( 1 ) )

start = timeit.default_timer()
util.Helpers.burnIn( view, img )
stop = timeit.default_timer()
print( 'Runtime for converting every pixel:', stop - start )
print( 'Image size:', arr.shape )

Runtime for converting every pixel: 22.912966964002408
Image size: (282, 330, 3)


## Pure Java Converter

In [8]:
converter_code = """
import net.imglib2.converter.Converter;
import net.imglib2.type.numeric.ARGBType;
import net.imglib2.type.numeric.integer.UnsignedByteType;
import net.imglib2.view.composite.Composite;

public class CompositeARGBTypeConverter< C extends Composite< UnsignedByteType > > implements Converter< C, ARGBType > {
    
    public void convert( C source, ARGBType target ) {
        target.set( ARGBType.rgba( source.get( 0 ).get(), source.get( 1 ).get(), source.get( 2 ).get(), 255 ) );
    }
    
}
"""

fp = pathlib.Path( tmp_dir ) / 'CompositeARGBTypeConverter.java'
print( tmp_dir )
with open( fp, 'w' ) as f:
    f.write( converter_code )

javac = pathlib.Path( os.environ[ 'JAVA_HOME' ] ) / 'bin' / 'javac'
proc = subprocess.run( 
    [ javac, '-cp', jnius_config.split_char.join( jnius_config.get_classpath() ), fp ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
if proc.returncode != 0:
    print ( proc.stderr )

/tmp/tmpq50_n63d


In [9]:
Converters = autoclass( 'net.imglib2.converter.Converters' )
converter = autoclass( 'CompositeARGBTypeConverter' )()
composite = util.Views.collapseReal( rai )
view = Converters.convert( cast( 'net.imglib2.RandomAccessibleInterval', composite ), converter, ARGBType() )
img = autoclass( 'net.imglib2.img.array.ArrayImgs' ).argbs( view.dimension( 0 ), view.dimension( 1 ) )

start = timeit.default_timer()
util.Helpers.burnIn( view, img )
stop = timeit.default_timer()
print( 'Runtime for converting every pixel:', stop - start )
print( 'Image size:', arr.shape )

Runtime for converting every pixel: 0.022073452000768157
Image size: (282, 330, 3)


In [10]:
bdv = util.BdvFunctions.show( view, 'argb', util.BdvOptions.options().is2D() )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel(), lambda : print( 'Different exit message!' ) )

Different exit message!


## BDV OverlayRenderer
`imglyb.util.GenericOverlayRenderer` implements `net.imglib2.ui.OverlayRenderer`. The `GenericOverlayRenderer` constructor accepts two optional arguments, `draw_overlays` and `set_canvas_size`, that implement the respective methods in the `OverlayRenderer` interface. If not specified, these methods will just return without any action. It is also possible to pass stateful functions (see example below).

In [11]:
# stateless overlay renderer
hello_world_color= autoclass( 'java.awt.Color' ).WHITE
def hello_world( g ):
    g2d = cast( 'java.awt.Graphics2D', g )
    g2d.setColor( hello_world_color )
    g2d.drawString("Hello world!", 30, 130 )
hello_world_overlay = util.GenericOverlayRenderer( hello_world )

# stateful overlay renderer with class that holds state
class RectangleOverlayRenderer:
    def __init__( self ):
        self.w = 0
        self.h = 0
        self.color= autoclass('java.awt.Color').WHITE
        self.stroke = autoclass('java.awt.BasicStroke')( 10 )

    def draw_overlays( self, g ):
        g2d = cast('java.awt.Graphics2D', g)
        g2d.setColor( self.color )
        g2d.setStroke( self.stroke )
        g2d.drawRect( self.w // 2 - self.w // 6, self.h // 2 - self.h // 6, self.w // 3, self.h // 3 )

    def set_canvas_size( self, width, height ):
        print( "Setting canvas size", width, height )
        self.w = width
        self.h = height
    
rectangle_state = RectangleOverlayRenderer()
rectangle_renderer = util.GenericOverlayRenderer( lambda g : rectangle_state.draw_overlays( g ), lambda w, h : rectangle_state.set_canvas_size( w, h ) )

## MouseMotionListener
In analogy to `GenericOverlayRenderer`, `imglyb.util.GenericMouseMotionListener` implements `java.awt.event.MouseMotionListener` with functions `mouse_dragged` and `mouse_moved` that implement the according interface methods and can be passed to the `GenericMouseMotionListener` constructor (default to no-op).

In [12]:
coordinate_printer = util.GenericMouseMotionListener(
        lambda e : print( "mouse dragged", e.getX(), e.getY() ),
        lambda e : print( "mouse moved", e.getX(), e.getY() )
        )

In [13]:
random = np.random.randint( 2**32, size=(300,200,100), dtype=np.uint32 )
bdv = util.BdvFunctions.show( imglyb.to_imglib_argb( random ), 'random' )
viewer = bdv.getBdvHandle().getViewerPanel()
viewer.getDisplay().addMouseMotionListener( coordinate_printer )
viewer.getDisplay().addOverlayRenderer( hello_world_overlay )
viewer.getDisplay().addOverlayRenderer( rectangle_renderer )

Setting canvas size 800 579
mouse moved 799 518
mouse moved 768 515
mouse moved 749 513
mouse moved 727 509
mouse moved 704 509
mouse moved 689 509
mouse moved 672 503
mouse moved 655 500
mouse moved 649 498
mouse moved 644 494
mouse moved 640 491
mouse moved 640 492
mouse moved 639 493
mouse moved 639 496
mouse moved 639 498
mouse moved 639 500
mouse moved 639 503
mouse moved 640 504
mouse moved 641 504
mouse moved 643 505
mouse moved 645 507
mouse moved 647 508
mouse moved 651 509
mouse moved 654 510
mouse moved 660 510
mouse moved 664 510
mouse moved 668 510
mouse moved 670 509
mouse moved 671 506
mouse moved 671 503
mouse moved 671 501
mouse moved 666 497
mouse moved 656 494
mouse moved 647 486
mouse moved 636 476
mouse moved 624 472
mouse moved 607 463
mouse moved 592 454
mouse moved 581 446
mouse moved 573 439
mouse moved 566 431
mouse moved 559 424
mouse moved 556 417
mouse moved 550 410
mouse moved 547 404
mouse moved 544 399
mouse moved 540 395
mouse moved 537 389
mouse moved 

Setting canvas size 1022 579
Setting canvas size 1033 579
Setting canvas size 1039 579
Setting canvas size 1040 579
Setting canvas size 1055 579
Setting canvas size 1070 579
Setting canvas size 1073 579
Setting canvas size 1082 579
mouse moved 1080 477
mouse moved 1071 477
mouse moved 1066 478
mouse moved 1067 477
mouse moved 1068 476
mouse moved 1069 475
mouse moved 1070 475
mouse moved 1071 474
mouse moved 1071 477
mouse moved 1070 480
mouse moved 1070 484
mouse moved 1070 488
mouse moved 1070 490
mouse moved 1071 492
mouse moved 1072 494
mouse moved 1074 498
mouse moved 1076 500
mouse moved 1078 502
mouse moved 1080 503
Setting canvas size 1073 579
Setting canvas size 954 579
Setting canvas size 865 579
Setting canvas size 819 579
Setting canvas size 795 579
Setting canvas size 774 579
Setting canvas size 765 579
Setting canvas size 762 579
Setting canvas size 765 579
Setting canvas size 820 579
Setting canvas size 896 579
Setting canvas size 936 579
Setting canvas size 975 579
Sett

mouse dragged 770 290
mouse dragged 769 290
mouse dragged 769 290
mouse dragged 770 290
mouse dragged 772 290
mouse dragged 774 290
mouse dragged 776 290
mouse dragged 777 290
mouse dragged 778 290
mouse dragged 780 290
mouse dragged 781 290
mouse dragged 782 290
mouse dragged 783 290
mouse dragged 784 290
mouse dragged 785 290
mouse dragged 786 290
mouse dragged 787 290
mouse dragged 788 290
mouse dragged 788 289
mouse moved 788 289
mouse moved 788 290
mouse moved 788 291
mouse moved 788 292
mouse moved 788 293
mouse moved 788 294
mouse moved 788 295
mouse moved 788 295
mouse moved 790 295
mouse moved 792 293
mouse moved 796 290
mouse moved 799 289
mouse moved 800 286
mouse moved 801 282
mouse moved 804 278
mouse moved 805 274
mouse moved 805 270
mouse moved 805 264
mouse moved 806 262
mouse moved 806 259
mouse moved 806 258
mouse moved 806 257
mouse moved 806 256
mouse moved 805 255
mouse dragged 800 258
mouse dragged 797 260
mouse dragged 792 261
mouse dragged 788 264
mouse dragged 

mouse moved 874 185
mouse moved 888 180
mouse moved 895 178
mouse moved 902 178
mouse moved 906 178
mouse moved 908 178
mouse moved 910 180
mouse moved 913 183
mouse moved 914 184
mouse moved 912 183
mouse moved 908 183
mouse moved 904 181
mouse moved 898 179
mouse moved 897 176
mouse moved 895 174
mouse moved 894 170
mouse moved 891 168
mouse moved 889 163
mouse moved 884 161
mouse moved 880 158
mouse moved 875 156
mouse moved 868 156
mouse moved 856 156
mouse moved 850 154
mouse moved 844 154
mouse moved 841 154
mouse moved 840 159
mouse moved 839 162
mouse moved 835 165
mouse moved 831 169
mouse moved 823 177
mouse moved 810 185
mouse moved 797 193
mouse moved 778 203
mouse moved 763 211
mouse moved 745 218
mouse moved 732 227
mouse moved 715 235
mouse moved 706 244
mouse moved 698 248
mouse moved 693 250
mouse moved 687 255
mouse moved 684 257
mouse moved 680 259
mouse moved 673 260
mouse moved 666 262
mouse moved 657 262
mouse moved 653 262
mouse moved 648 262
mouse moved 642 262


In [14]:
RealPoint = autoclass( 'net.imglib2.RealPoint' )

class Painter( PythonJavaClass ):
    __javainterfaces__ = ['org/scijava/ui/behaviour/DragBehaviour']


    def __init__( self, img, mask, viewer, paint_listener = lambda : None ):
        super( Painter, self ).__init__()
        self.img = img
        self.mask = mask
        self.radius = int( mask.shape[0] / 2 )
        self.viewer = viewer
        self.oX = 0
        self.oY = 0
        self.n_dim = len( img.shape )
        self.labelLocation = RealPoint( 3 )
        self.lower = np.empty( ( self.n_dim, ), dtype=np.int32 )
        self.upper = np.empty( ( self.n_dim, ), dtype=np.int32 )
        self.paint_listener = paint_listener

    @java_method('(II)V')
    def init( self, x, y ):
        self._paint( x, y )
        self.oX = x
        self.oY = y
        self.viewer.requestRepaint()

    @java_method('(II)V')
    def drag( self, x, y ):
        self._setCoordinates( self.oX, self.oY )
        n_dim = self.labelLocation.numDimensions()
        origin = np.array( [ self.labelLocation.getDoublePosition( d ) for d in range( n_dim ) ] )
        origin_p = RealPoint( n_dim )
        for d, p in enumerate( origin ):
            origin_p.setPosition( p, d )
        self._setCoordinates( x, y )
        target = np.array( [ self.labelLocation.getDoublePosition( d ) for d in range( n_dim ) ] )
        diff = target - origin
        length = np.linalg.norm( diff )
        direction = diff / length
        try:
            for l in range( 1, math.ceil( length ) ):
                for d, dist in enumerate( direction ):
                    origin_p.move( dist, d )
                self._paint_at_localizable( origin_p )
        except Exception as e:
            print( e )
            raise e

        self.oX = x
        self.oY = y
        self.viewer.requestRepaint()

    @java_method('(II)V')
    def end( self, x, y ):
        self.paint_listener()

    def _paint( self, x, y ):
        self._setCoordinates( x, y )
        self._paint_at_localizable( self.labelLocation )

    def _paint_at_localizable( self, labelLocation ):
        for d in range( self.n_dim ):
            int_pos = int( round( labelLocation.getDoublePosition( d ) ) )
            if int_pos < 0 or int_pos >= self.img.shape[ ::-1 ][ d ]:
                return
            self.lower[ d ] = int_pos - self.radius
            self.upper[ d ] = int_pos + self.radius
            
        self.lower = self.lower[::-1]
        self.upper = self.upper[::-1]

        img_lower = np.maximum( self.lower, 0 )
        img_upper = np.minimum( self.upper, self.img.shape )
        
        if np.any( img_lower >= img_upper ):
            return

        mask_lower = np.abs( np.minimum( self.lower, 0 ) ).astype( self.lower.dtype )
        mask_upper = np.minimum( mask_lower + ( img_upper - img_lower ), mask_lower + np.array( self.mask.shape ) )
        img_selection = tuple( slice(l, u) for l, u in zip( img_lower, img_upper ) )
        mask_selection = tuple( slice(l, u) for l, u in zip( mask_lower, mask_upper ) )

        # color_channels = tuple( 255 << ( ( idx * 8 ) if np.random.rand() > 0.5 else 0 ) for idx in range( self.n_dim )  )
        color_channels = tuple( np.random.randint( 128, 255 ) << ( idx * 8 ) for idx in range( self.n_dim )  )
        color = 0
        for c in color_channels:
            color = color | c
        try:
            self.img[ img_selection  ][ self.mask[ mask_selection ] ] = color
        except Exception as e:
            print( "EXCEPTION", e )
            raise e

    def _setCoordinates( self, x, y ):
        self.labelLocation.setPosition( x, 0 )
        self.labelLocation.setPosition( y, 1 )
        self.labelLocation.setPosition( 0, 2 )
        self.viewer.displayToGlobalCoordinates( self.labelLocation )

initial_img = np.zeros( ( 300, 200, 100 ), dtype=np.uint32 ) + ( 80 | ( 80 << 8 ) )
initial_rai = imglyb.to_imglib_argb( initial_img )
bdv = util.BdvFunctions.show( initial_rai, 'canvas' )
mask = np.ones( ( 10, 10, 10 ) ) == 1
painter = Painter( initial_img, mask, bdv.getBdvHandle().getViewerPanel() )

print( initial_img.shape, [ initial_rai.dimension( d ) for d in range( initial_rai.numDimensions() ) ] )

behaviors = util.Helpers.behaviours()
behaviors.install( bdv.getBdvHandle().getTriggerbindings(), "paint" )
behaviors.behaviour( painter, "paint", "SPACE button1" )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel() )


(300, 200, 100) [100, 200, 300]


/home/phil/.conda/envs/imglyb/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


Exited!


In [15]:
import h5py
import os
import pathlib
import urllib.request
import shutil

# data from https://emdatasets.com/category/emd-examples/
# Hexagonal Silicon Nitride
url = 'https://cremi.org/static/data/sample_A%2B_20160601.hdf'
path = pathlib.Path( '/tmp/sample_A+.h5' )
if not path.is_file():
    with urllib.request.urlopen( url ) as response, open( path, 'wb' ) as out_file:
        shutil.copyfileobj( response, out_file )
    print( 'successfully downloaded file!' )

with h5py.File( path, 'r' ) as f:
    data = f['volumes/raw'].value
rai = imglyb.to_imglib( data )
bdv = util.BdvFunctions.show( rai, 'rai', util.BdvOptions.options().sourceTransform( 4, 4, 40 ) )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel() )

successfully downloaded file!
Exited!
